# This notebook adds the manually created missing fare inputs in GTFS format

The missing fare include:
1. GG Transit
2. AC Transit
3. CCTA 3 routes
4. Marguerite
5. SF Bay Ferry
6. Muni 1 route
7. TriDelta
8. Napa VINE
9. Wheels

and correction to VTA fare.

In [1]:
import os
import sys
import yaml
import pickle
import glob

import pandas as pd

from network_wrangler import RoadwayNetwork
from network_wrangler import TransitNetwork
from network_wrangler import ProjectCard
from network_wrangler import Scenario
from network_wrangler import WranglerLogger

from lasso import ModelRoadwayNetwork
from lasso import StandardTransit
from lasso import Parameters
from lasso import mtc

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import logging
logger = logging.getLogger("WranglerLogger")
logger.handlers[0].stream = sys.stdout
logger.setLevel(logging.INFO)

In [4]:
# where user has Lasso installed
lasso_dir = "Z:/Data/Users/Sijia/MTC/github/Lasso" 

input_dir = "../data"
input_dir = "D:/github/travel-model-two-networks/data"
output_dir = os.path.join(input_dir, "interim", "step6_gtfs")

In [5]:
parameters = Parameters(lasso_base_dir = lasso_dir)

2021-02-26 12:33:30, INFO: Lasso base directory set as: Z:/Data/Users/Sijia/MTC/github/Lasso
2021-02-26 12:33:30, INFO: Lasso base directory set as: Z:/Data/Users/Sijia/MTC/github/Lasso
2021-02-26 12:33:30, INFO: Lasso base directory set as: Z:/Data/Users/Sijia/MTC/github/Lasso
2021-02-26 12:33:30, INFO: Lasso base directory set as: Z:/Data/Users/Sijia/MTC/github/Lasso


# read transit net

In [6]:
transit_net = TransitNetwork.read(
    feed_path = os.path.join(input_dir, "processed", "version_03")
)

2021-02-26 12:33:31, INFO: Read in transit feed from: D:/github/travel-model-two-networks/data\processed\version_03
2021-02-26 12:33:31, INFO: Read in transit feed from: D:/github/travel-model-two-networks/data\processed\version_03
2021-02-26 12:33:31, INFO: Removing calendar.txt from transit network config because file not found
2021-02-26 12:33:31, INFO: Removing calendar.txt from transit network config because file not found
2021-02-26 12:33:31, INFO: Removing calendar_dates.txt from transit network config because file not found
2021-02-26 12:33:31, INFO: Removing calendar_dates.txt from transit network config because file not found
2021-02-26 12:33:34, INFO: Removing transfers.txt from transit network config because file not found
2021-02-26 12:33:34, INFO: Removing transfers.txt from transit network config because file not found
2021-02-26 12:33:34, INFO: Removing feed_info.txt from transit network config because file not found
2021-02-26 12:33:34, INFO: Removing feed_info.txt fro

# read 2015 existing GTFS fare files

In [7]:
fare_attributes_df = pd.read_csv(
    os.path.join(input_dir,"interim","step6_gtfs","consolidated_gtfs_input","fare_attributes.txt"))
fare_rules_df = pd.read_csv(
    os.path.join(input_dir,"interim","step6_gtfs","consolidated_gtfs_input","fare_rules.txt"))

In [8]:
fare_attributes_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 742 entries, 0 to 741
Data columns (total 8 columns):
fare_id              742 non-null object
price                742 non-null float64
currency_type        742 non-null object
payment_method       742 non-null int64
transfers            44 non-null object
transfer_duration    264 non-null float64
agency_raw_name      742 non-null object
agency_id            250 non-null float64
dtypes: float64(3), int64(1), object(4)
memory usage: 46.5+ KB


In [9]:
fare_rules_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3250 entries, 0 to 3249
Data columns (total 7 columns):
fare_id              3250 non-null object
route_id_original    646 non-null object
agency_raw_name      3250 non-null object
origin_id            2818 non-null object
destination_id       2818 non-null object
contains_id          19 non-null object
route_id             538 non-null float64
dtypes: float64(1), object(6)
memory usage: 177.9+ KB


In [10]:
fare_attributes_df.agency_raw_name.value_counts().sort_index()

ACE_2017_3_20                                     17
BART_2015_8_3                                    152
Blue&Gold_gtfs_10_4_2017                           3
CCTA_2015_8_11                                     3
Caltrain_2015_5_13                                 7
Capitol_2017_3_20                                290
Emeryville_2016_10_26                              1
Fairfield_2015_10_14                               3
MVGo_2016_10_26                                    1
MarinTransit_2015_8_31                             2
RioVista_2015_8_20                                 2
SFMTA_2015_8_11                                    2
SamTrans_2015_8_20                                 4
SantaRosa_google_transit_08_28_15                  1
Soltrans_2016_5_20                                 2
SonomaCounty_2015_8_18                           237
Union_City_Transit_Aug-01-2015 to Jun-30-2017      1
VTA_2015_8_27                                     10
petalumatransit-petaluma-ca-us__11_12_15      

In [11]:
fare_rules_df.agency_raw_name.value_counts().sort_index()

ACE_2017_3_20                                 17
BART_2015_8_3                               2025
Blue&Gold_gtfs_10_4_2017                       3
CCTA_2015_8_11                               102
Caltrain_2015_5_13                           109
Capitol_2017_3_20                            290
Fairfield_2015_10_14                          33
MarinTransit_2015_8_31                        19
RioVista_2015_8_20                             4
SFMTA_2015_8_11                               83
SamTrans_2015_8_20                            86
Soltrans_2016_5_20                            14
SonomaCounty_2015_8_18                       237
VTA_2015_8_27                                212
petalumatransit-petaluma-ca-us__11_12_15       1
westcat-ca-us_9_17_2015                       15
Name: agency_raw_name, dtype: int64

# drop special fare in VTA

In [12]:
vta_fare_rules_df = pd.read_csv(
    os.path.join(input_dir,"external","gtfs","add_fare","VTA_fare","fare_rules.txt")
)

vta_fare_attributes_df = pd.read_csv(
    os.path.join(input_dir,"external","gtfs","add_fare","VTA_fare","fare_attributes.txt")
)

vta_fare_rules_df["agency_raw_name"] = "VTA_2015_8_27"
vta_fare_attributes_df["agency_raw_name"] = "VTA_2015_8_27"

In [13]:
fare_rules_df = pd.concat(
    [
        fare_rules_df[fare_rules_df.agency_raw_name != "VTA_2015_8_27"],
        vta_fare_rules_df
    ],
    sort = False,
    ignore_index = True
)

fare_attributes_df = pd.concat(
    [
        fare_attributes_df[fare_attributes_df.agency_raw_name!="VTA_2015_8_27"],
        vta_fare_attributes_df
    ],
    sort = False,
    ignore_index = True
)

In [14]:
fare_attributes_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 735 entries, 0 to 734
Data columns (total 8 columns):
fare_id              735 non-null object
price                735 non-null float64
currency_type        735 non-null object
payment_method       732 non-null float64
transfers            40 non-null object
transfer_duration    262 non-null float64
agency_raw_name      735 non-null object
agency_id            250 non-null float64
dtypes: float64(4), object(4)
memory usage: 46.1+ KB


In [15]:
fare_rules_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3129 entries, 0 to 3128
Data columns (total 7 columns):
fare_id              3129 non-null object
route_id_original    434 non-null object
agency_raw_name      3129 non-null object
origin_id            2818 non-null object
destination_id       2818 non-null object
contains_id          19 non-null object
route_id             446 non-null float64
dtypes: float64(1), object(6)
memory usage: 171.2+ KB


# add GG Transit

In [17]:
ggtransit_fare_df = pd.read_csv(
    os.path.join(input_dir,"external","gtfs","add_fare","GGTransit_fare","fare.txt")
)

In [18]:
ggtransit_fare_df

,bus zone,1,2,3,4,5,6,7
0,1,4.75,5.25,6.50,7.75,11.25,12.50,9.75
1,2,5.25,2.00,2.00,2.00,7.75,9.25,5.25
2,3,6.50,2.00,2.00,2.00,6.50,7.75,5.25
3,4,7.75,2.00,2.00,2.00,5.25,6.50,5.25
4,5,11.25,7.75,6.50,5.25,4.75,4.75,9.75
5,6,12.50,9.25,7.75,6.50,4.75,4.75,9.75
6,7,9.75,5.25,5.25,5.25,9.75,9.75,4.75


In [19]:
ggtransit_fare_df = pd.melt(ggtransit_fare_df, 
                            id_vars=['bus zone'], 
                            value_vars=["1","2","3","4","5","6","7"],
                            var_name='destination_id', 
                            value_name="price")

ggtransit_fare_df.rename(columns = {"bus zone" : "origin_id"},
                         inplace = True)

ggtransit_fare_df["fare_id"] = range(1, 1+len(ggtransit_fare_df))

ggtransit_fare_df["agency_raw_name"] = "GGTransit_2015_9_3"

In [20]:
fare_attributes_df = pd.concat(
    [
        fare_attributes_df,
        ggtransit_fare_df[["fare_id", "price", "agency_raw_name"]]
    ],
    sort = False,
    ignore_index = True
)

fare_rules_df = pd.concat(
    [
        fare_rules_df,
        ggtransit_fare_df[
            ["fare_id", "origin_id", "destination_id", "agency_raw_name"]]
    ],
    sort = False,
    ignore_index = True
)

In [21]:
fare_attributes_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 784 entries, 0 to 783
Data columns (total 8 columns):
fare_id              784 non-null object
price                784 non-null float64
currency_type        735 non-null object
payment_method       732 non-null float64
transfers            40 non-null object
transfer_duration    262 non-null float64
agency_raw_name      784 non-null object
agency_id            250 non-null float64
dtypes: float64(4), object(4)
memory usage: 49.1+ KB


In [22]:
fare_rules_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3178 entries, 0 to 3177
Data columns (total 7 columns):
fare_id              3178 non-null object
route_id_original    434 non-null object
agency_raw_name      3178 non-null object
origin_id            2867 non-null object
destination_id       2867 non-null object
contains_id          19 non-null object
route_id             446 non-null float64
dtypes: float64(1), object(6)
memory usage: 173.9+ KB


# add AC Transit

In [23]:
actransit_fare_rules_df = pd.read_csv(
    os.path.join(input_dir,"external","gtfs","add_fare","ACTransit_fare","fare_rules.txt")
)

actransit_fare_attributes_df = pd.read_csv(
    os.path.join(input_dir,"external","gtfs","add_fare","ACTransit_fare","fare_attributes.txt")
)

actransit_fare_rules_df["agency_raw_name"] = "ACTransit_2015_8_14"
actransit_fare_attributes_df["agency_raw_name"] = "ACTransit_2015_8_14"

In [24]:
actransit_fare_rules_df.rename(columns = {"route_id":"route_id_original"},
                               inplace = True)

actransit_fare_rules_df = pd.merge(
    actransit_fare_rules_df,
    transit_net.feed.routes[["route_id_original", "route_id", "agency_raw_name"]],
    how = "left",
    on = ["route_id_original", "agency_raw_name"]
)

In [25]:
fare_attributes_df = pd.concat(
    [
        fare_attributes_df,
        actransit_fare_attributes_df
    ],
    sort = False,
    ignore_index = True
)

fare_rules_df = pd.concat(
    [
        fare_rules_df,
        actransit_fare_rules_df
    ],
    sort = False,
    ignore_index = True
)

In [26]:
fare_attributes_df.info()
fare_rules_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 786 entries, 0 to 785
Data columns (total 8 columns):
fare_id              786 non-null object
price                786 non-null float64
currency_type        737 non-null object
payment_method       732 non-null float64
transfers            40 non-null object
transfer_duration    262 non-null float64
agency_raw_name      786 non-null object
agency_id            250 non-null float64
dtypes: float64(4), object(4)
memory usage: 49.2+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3333 entries, 0 to 3332
Data columns (total 7 columns):
fare_id              3333 non-null object
route_id_original    589 non-null object
agency_raw_name      3333 non-null object
origin_id            2867 non-null object
destination_id       2867 non-null object
contains_id          19 non-null object
route_id             598 non-null object
dtypes: object(7)
memory usage: 182.4+ KB


# add SF Bay Ferry

In [27]:
sfbay_fare_rules_df = pd.read_csv(
    os.path.join(input_dir,"external","gtfs","add_fare","SF_Bay_ferry_fare","fare_rules.txt"),
    dtype = {"route_id":"str"}
)

sfbay_fare_attributes_df = pd.read_csv(
    os.path.join(input_dir,"external","gtfs","add_fare","SF_Bay_ferry_fare","fare_attributes.txt")
)

sfbay_fare_rules_df["agency_raw_name"] = "SF_Bay_Ferry2016_07_01"
sfbay_fare_attributes_df["agency_raw_name"] = "SF_Bay_Ferry2016_07_01"

In [28]:
sfbay_fare_rules_df.rename(columns = {"route_id":"route_id_original"},
                           inplace = True)

sfbay_fare_rules_df = pd.merge(
    sfbay_fare_rules_df,
    transit_net.feed.routes[["route_id_original", "route_id", "agency_raw_name"]],
    how = "left",
    on = ["route_id_original", "agency_raw_name"]
)

In [29]:
fare_attributes_df = pd.concat(
    [
        fare_attributes_df,
        sfbay_fare_attributes_df
    ],
    sort = False,
    ignore_index = True
)

fare_rules_df = pd.concat(
    [
        fare_rules_df,
        sfbay_fare_rules_df
    ],
    sort = False,
    ignore_index = True
)

In [30]:
fare_attributes_df.info()
fare_rules_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 792 entries, 0 to 791
Data columns (total 8 columns):
fare_id              792 non-null object
price                792 non-null float64
currency_type        743 non-null object
payment_method       738 non-null float64
transfers            46 non-null object
transfer_duration    262 non-null float64
agency_raw_name      792 non-null object
agency_id            250 non-null float64
dtypes: float64(4), object(4)
memory usage: 49.6+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3339 entries, 0 to 3338
Data columns (total 7 columns):
fare_id              3339 non-null object
route_id_original    595 non-null object
agency_raw_name      3339 non-null object
origin_id            2867 non-null object
destination_id       2867 non-null object
contains_id          19 non-null object
route_id             602 non-null object
dtypes: object(7)
memory usage: 182.7+ KB


# add marguerite fare

In [31]:
marguerite_fare_attributes_df = pd.DataFrame(
    data = {"fare_id" : [1], "price" : [0], "agency_raw_name" : "Marguerite_2016_10_10"}
)

In [32]:
marguerite_fare_attributes_df

,fare_id,price,agency_raw_name
0,1,0,Marguerite_2016_10_10


In [33]:
fare_attributes_df = pd.concat(
    [
        fare_attributes_df,
        marguerite_fare_attributes_df
    ],
    sort = False,
    ignore_index = True
)

# add TriDelta fare

In [34]:
tridelta_fare_rules_df = pd.read_csv(
    os.path.join(input_dir,"external","gtfs","add_fare","TriDelta_fare","fare_rules.txt"),
    dtype = {"route_id":"str"}
)

tridelta_fare_attributes_df = pd.read_csv(
    os.path.join(input_dir,"external","gtfs","add_fare","TriDelta_fare","fare_attributes.txt")
)

tridelta_fare_rules_df["agency_raw_name"] = "TriDelta-GTFS-2018-05-24_21-43-17"
tridelta_fare_attributes_df["agency_raw_name"] = "TriDelta-GTFS-2018-05-24_21-43-17"

In [35]:
tridelta_fare_rules_df.rename(columns = {"route_id":"route_id_original"},
                           inplace = True)

tridelta_fare_rules_df = pd.merge(
    tridelta_fare_rules_df,
    transit_net.feed.routes[["route_id_original", "route_id", "agency_raw_name"]],
    how = "left",
    on = ["route_id_original", "agency_raw_name"]
)

In [36]:
fare_attributes_df = pd.concat(
    [
        fare_attributes_df,
        tridelta_fare_attributes_df
    ],
    sort = False,
    ignore_index = True
)

fare_rules_df = pd.concat(
    [
        fare_rules_df,
        tridelta_fare_rules_df
    ],
    sort = False,
    ignore_index = True
)

fare_attributes_df.info()
fare_rules_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 795 entries, 0 to 794
Data columns (total 8 columns):
fare_id              795 non-null object
price                795 non-null float64
currency_type        743 non-null object
payment_method       738 non-null float64
transfers            46 non-null object
transfer_duration    262 non-null float64
agency_raw_name      795 non-null object
agency_id            250 non-null float64
dtypes: float64(4), object(4)
memory usage: 49.8+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3358 entries, 0 to 3357
Data columns (total 7 columns):
fare_id              3358 non-null object
route_id_original    614 non-null object
agency_raw_name      3358 non-null object
origin_id            2867 non-null object
destination_id       2867 non-null object
contains_id          19 non-null object
route_id             616 non-null object
dtypes: object(7)
memory usage: 183.8+ KB


# add Vine fare

In [37]:
vine_fare_rules_df = pd.read_csv(
    os.path.join(input_dir,"external","gtfs","add_fare","Vine_fare","fare_rules.txt"),
    dtype = {"route_id":"str"}
)

vine_fare_attributes_df = pd.read_csv(
    os.path.join(input_dir,"external","gtfs","add_fare","Vine_fare","fare_attributes.txt")
)

vine_fare_rules_df["agency_raw_name"] = "Vine_GTFS_PLUS_2015"
vine_fare_attributes_df["agency_raw_name"] = "Vine_GTFS_PLUS_2015"

In [38]:
vine_fare_rules_df.rename(columns = {"route_id":"route_id_original"},
                           inplace = True)

vine_fare_rules_df = pd.merge(
    vine_fare_rules_df,
    transit_net.feed.routes[["route_id_original", "route_id", "agency_raw_name"]],
    how = "left",
    on = ["route_id_original", "agency_raw_name"]
)

In [39]:
fare_attributes_df = pd.concat(
    [
        fare_attributes_df,
        vine_fare_attributes_df
    ],
    sort = False,
    ignore_index = True
)

fare_rules_df = pd.concat(
    [
        fare_rules_df,
        vine_fare_rules_df
    ],
    sort = False,
    ignore_index = True
)

fare_attributes_df.info()
fare_rules_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 798 entries, 0 to 797
Data columns (total 8 columns):
fare_id              798 non-null object
price                798 non-null float64
currency_type        743 non-null object
payment_method       738 non-null float64
transfers            46 non-null object
transfer_duration    262 non-null float64
agency_raw_name      798 non-null object
agency_id            250 non-null float64
dtypes: float64(4), object(4)
memory usage: 50.0+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3371 entries, 0 to 3370
Data columns (total 7 columns):
fare_id              3371 non-null object
route_id_original    627 non-null object
agency_raw_name      3371 non-null object
origin_id            2867 non-null object
destination_id       2867 non-null object
contains_id          19 non-null object
route_id             629 non-null object
dtypes: object(7)
memory usage: 184.5+ KB


# add wheels fare

In [40]:
wheels_fare_attributes_df = pd.DataFrame(
    data = {"fare_id" : [1], 
            "price" : [2], 
            "agency_raw_name" : ["Wheels_2016_7_13"]}
)

In [41]:
wheels_fare_attributes_df

,fare_id,price,agency_raw_name
0,1,2,Wheels_2016_7_13


In [42]:
fare_attributes_df = pd.concat(
    [
        fare_attributes_df,
        wheels_fare_attributes_df
    ],
    sort = False,
    ignore_index = True
)

# add CCTA two routes that did not have fare in its GTFS

In [43]:
ccta_fare_rules_df = pd.read_csv(
    os.path.join(input_dir,"external","gtfs","add_fare","CCTA_fare","fare_rules.txt"),
    dtype = {"route_id":"str"}
)

ccta_fare_attributes_df = pd.read_csv(
    os.path.join(input_dir,"external","gtfs","add_fare","CCTA_fare","fare_attributes.txt")
)

ccta_fare_rules_df["agency_raw_name"] = "CCTA_2015_8_11"
ccta_fare_attributes_df["agency_raw_name"] = "CCTA_2015_8_11"

In [44]:
ccta_fare_rules_df.rename(columns = {"route_id":"route_id_original"},
                           inplace = True)

ccta_fare_rules_df = pd.merge(
    ccta_fare_rules_df,
    transit_net.feed.routes[["route_id_original", "route_id", "agency_raw_name"]],
    how = "left",
    on = ["route_id_original", "agency_raw_name"]
)

In [45]:
ccta_fare_rules_df

,fare_id,route_id_original,agency_raw_name,route_id
0,missing,14_merged_381003103,CCTA_2015_8_11,168
1,missing,25_merged_381003089,CCTA_2015_8_11,172
2,missing,26_merged_381003090,CCTA_2015_8_11,173


In [46]:
fare_attributes_df = pd.concat(
    [
        fare_attributes_df,
        ccta_fare_attributes_df
    ],
    sort = False,
    ignore_index = True
)

fare_rules_df = pd.concat(
    [
        fare_rules_df,
        ccta_fare_rules_df
    ],
    sort = False,
    ignore_index = True
)

fare_attributes_df.info()
fare_rules_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 800 entries, 0 to 799
Data columns (total 8 columns):
fare_id              800 non-null object
price                800 non-null float64
currency_type        743 non-null object
payment_method       738 non-null float64
transfers            46 non-null object
transfer_duration    262 non-null float64
agency_raw_name      800 non-null object
agency_id            250 non-null float64
dtypes: float64(4), object(4)
memory usage: 50.1+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3374 entries, 0 to 3373
Data columns (total 7 columns):
fare_id              3374 non-null object
route_id_original    630 non-null object
agency_raw_name      3374 non-null object
origin_id            2867 non-null object
destination_id       2867 non-null object
contains_id          19 non-null object
route_id             632 non-null object
dtypes: object(7)
memory usage: 184.6+ KB


# add muni one missing bus fare

In [47]:
sfmta_fare_rules_df = pd.read_csv(
    os.path.join(input_dir,"external","gtfs","add_fare","SFMTA_fare","fare_rules.txt"),
    dtype = {"route_id":"str"}
)

sfmta_fare_rules_df["agency_raw_name"] = "SFMTA_2015_8_11"

In [48]:
sfmta_fare_rules_df.rename(columns = {"route_id":"route_id_original"},
                           inplace = True)

sfmta_fare_rules_df = pd.merge(
    sfmta_fare_rules_df,
    transit_net.feed.routes[["route_id_original", "route_id", "agency_raw_name"]],
    how = "left",
    on = ["route_id_original", "agency_raw_name"]
)

fare_rules_df = pd.concat(
    [
        fare_rules_df,
        sfmta_fare_rules_df
    ],
    sort = False,
    ignore_index = True
)

fare_attributes_df.info()
fare_rules_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 800 entries, 0 to 799
Data columns (total 8 columns):
fare_id              800 non-null object
price                800 non-null float64
currency_type        743 non-null object
payment_method       738 non-null float64
transfers            46 non-null object
transfer_duration    262 non-null float64
agency_raw_name      800 non-null object
agency_id            250 non-null float64
dtypes: float64(4), object(4)
memory usage: 50.1+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3375 entries, 0 to 3374
Data columns (total 7 columns):
fare_id              3375 non-null object
route_id_original    631 non-null object
agency_raw_name      3375 non-null object
origin_id            2867 non-null object
destination_id       2867 non-null object
contains_id          19 non-null object
route_id             633 non-null object
dtypes: object(7)
memory usage: 184.7+ KB


# write out

In [49]:
fare_attributes_df.to_csv(os.path.join(output_dir, "fare_attributes.txt"),
                          sep = ",",
                          index = False)

fare_rules_df.to_csv(os.path.join(output_dir, "fare_rules.txt"),
                          sep = ",",
                          index = False)